# Feature selection and feature extraction
#### Part of the course on "Foundations of machine learning", Department of Mathematics and Statistics, University of Turku, Finland
#### Lectures available on YouTube: https://youtube.com/playlist?list=PLbkSohdmxoVAZ9DEHEWHjeGK7Ei-DjKHI&si=Msu74_I0qhLrRWcu
#### Code available on GitHub: https://github.com/ionpetre/FoundML_course_assignments

#### This notebook is based on the following sources: 
> https://www.kaggle.com/code/halflingwizard/feature-selection-from-600-to-17-features#5--Recursive-feature-elimination (MATT NAMVARPOUR)

> https://www.kaggle.com/code/parulpandey/penguin-dataset-the-new-iris (PARUL PANDEY)

> https://www.kaggle.com/code/pascaldupont/case-study-project-r-programming-language (PSCAL DUPONT)

> https://www.kaggle.com/code/arthurtok/interactive-intro-to-dimensionality-reduction (ANISOTROPIC)

> https://www.kaggle.com/code/jyotiprasadpal/dimension-reduction-methods (JYOTI PRASAD PAL)

> https://www.kaggle.com/code/ohseokkim/the-curse-of-dimensionality-dimension-reduction (OH SEOK KIM)

Feature selection and feature extraction are fundamental steps in machine learning, very important in model development and performance. Feature selection involves identifying and choosing the most relevant features from the dataset, aiming to reduce complexity, enhance model efficiency, and avoid overfitting. By selecting the right subset of features, the model also becomes more interpretable and computationally efficient, making it easier to comprehend and implement in real-world applications. On the other hand, feature extraction involves transforming the original data into a new set of features, often with reduced dimensionality while retaining essential information. This process enhances model performance by representing the data in a more meaningful and compact manner, facilitating better generalization and interpretation. Both feature selection and feature extraction are crucial strategies to streamline model complexity, improve predictive accuracy, and facilitate efficient decision-making in machine learning.

We demonstrate in this notebook the following methods:

I. Feature selection
 1. Percent Missing Values
 2. Ammount of Variation
 3. Pairwise Correlation
 4. Correlation with Target
 5. Recursive feature elimination
 6. Select from model

II. Feature extraction
 1. PCA
 2. LDA
 3. t-SNE

#### Load the libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, make_scorer, RocCurveDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, Normalizer

from yellowbrick.model_selection import RFECV
from yellowbrick.classifier import ClassPredictionError, ConfusionMatrix

# Import the 3 dimensionality reduction methods
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
# Reset the seed of the random number generator, for reproducibility purposes

import os

def reset_seed(SEED = 0):
    """Reset the seed for every random library in use (System, numpy)"""

    os.environ['PYTHONHASHSEED']=str(SEED)
    np.random.seed(SEED)


reset_seed(150)

#### Load the dataset: we use the UCI SECOM dataset (https://archive.ics.uci.edu/dataset/179/secom)
> "A complex modern semi-conductor manufacturing process is normally under consistent surveillance via the monitoring of signals/variables collected from sensors and or process measurement points. However, not all of these signals are equally valuable in a specific monitoring system. The measured signals contain a combination of useful information, irrelevant information as well as noise. It is often the case that useful information is buried in the latter two. Engineers typically have a much larger number of signals than are actually required. If we consider each type of signal as a feature, then feature selection may be applied to identify the most relevant signals. The Process Engineers may then use these signals to determine key factors contributing to yield excursions downstream in the process. This will enable an increase in process throughput, decreased time to learning and reduce the per unit production costs."


In [ ]:
from sklearn.datasets import fetch_openml

secom_X, _ = fetch_openml(
    data_id=43587,
    as_frame=True,
    return_X_y=True,
    parser = 'auto'
)

print(f'There are {secom_X.shape[0]} records with {secom_X.shape[1]} features!')
n_features0 = secom_X.shape[1]

#### Separate the data into features and target. 

In [ ]:
secom_y = secom_X[['Pass/Fail']]
secom_X = secom_X.drop(['Pass/Fail'], axis=1)

print('The UCI SECOM dataset:')
print(secom_X.info())

print(secom_X.head())

print('\n The labels:')
print(secom_y.info())

In [ ]:
# Check how imbalanced the dataset is 

secom_y.value_counts()

NOTE: The data is highly imbalanced, so we make the split train/validation/test in a stratified manner. 

In [ ]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    secom_X, 
    secom_y, 
    test_size=0.2, 
    random_state=2023, 
    stratify=secom_y,
    shuffle=True
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid, 
    y_train_valid, 
    test_size=0.25, 
    random_state=2023, 
    stratify=y_train_valid,
    shuffle=True
)

del X_train_valid
del y_train_valid

# convert to pandas dataframe
X_train = pd.DataFrame(X_train, columns=secom_X.columns)
X_valid = pd.DataFrame(X_valid, columns=secom_X.columns)
X_test = pd.DataFrame(X_test, columns=secom_X.columns)
y_train = pd.DataFrame(y_train, columns=secom_y.columns)
y_valid = pd.DataFrame(y_valid, columns=secom_y.columns)
y_test = pd.DataFrame(y_test, columns=secom_y.columns)

del secom_X
del secom_y

NOTE: We have 939 samples in training, 314 in each of validation and test, all with 592 features. 

#### Re-labeling the Target values
The labels are encoded in a slightly unusual way:
> -1 corresponds to a pass and 1 corresponds to a fail and the data time stamp is for that specific test point.

We change them to a more standard way so that each failure is is encoded as 0 while 1 corresponds to a pass (this of course is not crucial, it just avoid possible confusion later on). 

In [ ]:
y_train = y_train.replace(to_replace=[-1, 1], value=[1, 0])
y_valid = y_valid.replace(to_replace=[-1, 1], value=[1, 0])
y_test = y_test.replace(to_replace=[-1, 1], value=[1, 0])

#### Data exploration

In [ ]:
type_dct = {str(k): len(list(v)) for k, v in X_train.groupby(X_train.dtypes, axis=1)}
type_dct

Check the column which has `object` type.

In [ ]:
X_train.select_dtypes(include=['object'])

For simplicity, let's drop the 'Time' feature, it will not be important for the questions asked in this notebook. 

In [ ]:
X_train_notime = X_train.drop(['Time'], axis=1)
X_valid_notime = X_valid.drop(['Time'], axis=1)
X_test_notime = X_test.drop(['Time'], axis=1)

del X_train
del X_valid
del X_test

#### Count the missing values on each feature

In [ ]:
np.count_nonzero(X_train_notime.isna().sum())

So, 478/590 features contain missing values. Let's see the highest ammounts of missing values in a column:

In [ ]:
# A nice visualization of the missing values in a dataframe
# credit: https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction. 

def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
# Check the missing values in the dataset

X_train_missing= missing_values_table(X_train_notime)
print(X_train_missing.to_string())
del X_train_missing

Lots of missing values. We will have to deal with them in a similar way as done in the previous week's assignment https://github.com/ionpetre/FoundML_course_assignments/blob/main/FML1w5_incomplete_data.ipynb

### Evaluation model and metrics
Throughout this notebook we will deal with dimensionality reduction, i.e., with reducing the number of features in the dataset, using various methods. This will obviously lead to some loss of information/quality and it is important to quantify this loss, so we can judge when a dimensionality reduction step is worth taking. 

After each dimensionality reduction step we take we will train a classifier on the transformed dataset and we will measure its performance. The accuracy sore is not appropriate for imbalanced datasets. Instead we will use the F1 score, which is the harmonic mean between the precision and the recall. It takes values between 0 and 1, with 1 being a perfect fit, and 0 showing that either the precision or the recall are 0, so the model is quite poor in at least one of the classes. We also display the ROC-AUC curve (the further it is from the main diagonal towards the top-left corner, the better the model), and the confusion matrix (showing the number of points in each class that is missclassified). Finally, we also record the training time for each version of the dataset. 

We imputate the missing values separately before each such evaluation. This is because the imputation may be done differently, depending on the features still retained in the dataset (albeit this is not the case with the most frequent-based technique we use in this notebook).

For the classifier we will use a logistic regression (any other classifier would do). 

In [ ]:
def evaluate(train_df, test_df, train_target, test_target):
    
    # scale the data so logistic regression works better
    scaler = StandardScaler()
    scaler.fit(train_df)
    train_std = pd.DataFrame(scaler.transform(train_df), columns=train_df.columns)
    test_std = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)
    
    # training the model
    logreg = LogisticRegression(
        random_state = 175, 
        class_weight='balanced', 
        C=200, 
        dual=False, 
        solver='liblinear'
    )
    start_time = datetime.datetime.now()
    logreg.fit(train_std, train_target.values.ravel())
    elapsed = datetime.datetime.now() - start_time
    time = int(elapsed.total_seconds()*1000)
    
    logreg.class_counts_ = train_target.nunique()
    
    # evaluation and scoring
    y_pred = logreg.predict(test_std)
    y_true = test_target.values.ravel()
    f1score = f1_score(y_true, y_pred, average='micro')
    roc_aucscore = roc_auc_score(y_true, y_pred, average='micro')
    
    # visualizations
    cre = ClassPredictionError(
        logreg, 
        isfitted=True,
        classes=['fail', 'pass'],
        label_encoder={0: 'fail', 1: 'pass'},
        size=(400, 400)
    )
    cre.score(test_std, y_true)
    cre.show()
    cm = ConfusionMatrix(
        logreg, 
        isfitted=True,
        classes=['fail', 'pass'],
        label_encoder={0: 'fail', 1: 'pass'},
        percent= True,
        size=(300, 300)
    )
    
    cm.score(test_std, y_true)
    cm.show()
    #rocauc = plot_roc_curve(logreg, test_std, y_true)
    RocCurveDisplay.from_estimator(logreg, test_std, y_true)
    plt.show()
    
    return time, f1score, roc_aucscore

We check our baseline: how is the full dataset learned by our model. 

In [ ]:
# lists to record time and scores

f1scores = []
roc_aucscores = []
times = []
n_features = []

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_notime)
X_imputed_train = pd.DataFrame(imputer.transform(X_train_notime), columns = X_train_notime.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_notime), columns = X_valid_notime.columns)

# lists to record time and scores

time, f1score, roc_aucscore = evaluate(train_df = X_imputed_train, 
                                       test_df = X_imputed_valid, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
n_features.append(len(X_imputed_train.columns))

del X_imputed_train
del X_imputed_valid
del time
del f1score
del roc_aucscore

Even though the F1 score appears quite high, we can see that the model struggles to learn the data from the ROC-AUC cruves (the score is much closer to 0.5 than to 1) and from the confusion matrix (typical situation for imbalanced data: the model learns the larger class much better than the smaller class). 

## I. Feature Selection

### I.1. Percent Missing Values
We remove the columns that have most of their data missing. Imputing the missing values is not a good option because there is too little data to base the imputation on. We remove the columns missing more than 50% of their data. 

In [ ]:
def percentna(dataframe, threshold):
    columns = dataframe.columns[(dataframe.isna().sum()/dataframe.shape[1])>threshold]
    return columns.tolist()

Now I'm going to use this function to discover columns with more than 50% of their data missing. 

In [ ]:
sparse_columns = percentna(X_train_notime, 0.5)
X_train_nosparse = X_train_notime.drop(sparse_columns, axis=1)
X_valid_nosparse = X_valid_notime.drop(sparse_columns, axis=1)

n_features1 = X_train_nosparse.shape[1]
print(f'From {len(X_train_notime.columns)} features, we are left with {n_features1} features')
n_features.append(n_features1)

#### Evaluate the effect of removing the sparse features

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_nosparse)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_nosparse), columns = X_train_nosparse.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_nosparse), columns = X_valid_nosparse.columns)

time, f1score, roc_aucscore = evaluate(train_df = X_imputed_train, 
                                       test_df = X_imputed_valid, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)

del X_imputed_train
del X_imputed_valid
del time
del f1score
del roc_aucscore

The situation is roughly the same. The model trained a little faster. 

### I.2. Ammount of variation
We remove the features with very low variance, i.e., those that are almost constant in every row. In fact, to be conservative, we only eliminate those with variance 0, i.e., the constant features. 

In [ ]:
# Note: this methods does accept N/A values, no need to use imputation before applying it. 

selector = VarianceThreshold(threshold = 0.0)
selector.fit(X_train_nosparse)

mask = selector.get_support()
columns = X_train_nosparse.columns
selected_cols = columns[mask]
n_features2 = len(selected_cols)
print(f'From {len(columns)} features, we are left with {n_features2} features')
n_features.append(n_features2)

We remove these columns from the train/valid/test datasets. 

In [ ]:
X_train_var = pd.DataFrame(selector.transform(X_train_nosparse), columns = selected_cols)
X_valid_var = pd.DataFrame(selector.transform(X_valid_nosparse), columns = selected_cols)

#### Evaluate the effect of removing the constant features

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_var)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_var), columns = X_train_var.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_var), columns = X_valid_var.columns)

time, f1score, roc_aucscore = evaluate(train_df = X_imputed_train, 
                                       test_df = X_imputed_valid, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)

del X_imputed_train
del X_imputed_valid
del time
del f1score
del roc_aucscore

The performance of the model is about the same, but it trained faster. 

### I.3. Pairwise Correlation

The correlation coefficient has values between -1 to 1
- A value closer to 0 implies weaker correlation (exact 0 implying no correlation)
- A value closer to 1 implies stronger positive correlation
- A value closer to -1 implies stronger negative correlation

If two independent features (independent = not target!) have a high absolute correlation, the information they offer for our ML model is basically the same. Correlated features in general don't improve models, so we can drop either one of them, because they are redundant.

In [ ]:
# We use the following function which is originally written by 
# [Krish C Naik](https://github.com/krishnaik06/Complete-Feature-Selection/blob/master/2-Feature%20Selection-%20Correlation.ipynb).

# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything other feature

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(X_train_var, 0.95)
X_train_corr = X_train_var.drop(corr_features, axis=1)
X_valid_corr = X_valid_var.drop(corr_features, axis=1)
n_features3 = X_train_corr.shape[1]

print(f'From {len(X_train_var.columns)} features, we are left with {n_features3} features')
n_features.append(n_features3)

#### Evaluate the effect of removing the constant features

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_corr)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_corr), columns = X_train_corr.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_corr), columns = X_valid_corr.columns)

time, f1score, roc_aucscore = evaluate(train_df = X_imputed_train, 
                                       test_df = X_imputed_valid, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)

del X_imputed_train
del X_imputed_valid
del time
del f1score
del roc_aucscore

The situation is still about the same, but the model trained even faster. 

### I.4. Correlation with Target
We would like our features to have high correlation with the target. if a feature has low correlation with target, it means that it is not a helpful feature for predicting the target, and hence, it should be removed. The following function will calculate correlation of each feature with the target, and then returns the columns that have a correlation less than the chosen threshold.

In [ ]:
def corrwith_target(dataframe, target, threshold):
    cor = dataframe.corr()
    #Correlation with output variable
    cor_target = abs(cor[target])
    #Selecting non correlated features
    relevant_features = cor_target[cor_target<threshold]
    return relevant_features.index.tolist()[:-1]

In [ ]:
# in order to find the correlation with target, we need to add target as a column to X_train_corr
dummy_train = X_train_corr.copy(deep = True)
dummy_train['target'] = y_train

corrwith_cols = corrwith_target(dummy_train, 'target', 0.05)
X_train_corw = X_train_corr.drop(corrwith_cols, axis=1)
X_valid_corw = X_valid_corr.drop(corrwith_cols, axis=1)
n_features4 = X_train_corw.shape[1]

print(f'From {len(X_train_corr.columns)} features, we are left with {n_features4} features')
n_features.append(n_features4)

#### Evaluate the effect of removing the constant features

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_corw)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_corw), columns = X_train_corw.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_corw), columns = X_valid_corw.columns)

time, f1score, roc_aucscore = evaluate(train_df = X_imputed_train, 
                                       test_df = X_imputed_valid, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)

del X_imputed_train
del X_imputed_valid
del time
del f1score
del roc_aucscore

The model pays more attention to the smaller class (more of those are classified correctly), but with quite a drastic change in quality. And it trains faster still, of course. 

### I.5. Recursive feature elimination
We use Recursive feature elimination with cross-validation to find the optimum number of features from the remaining 38 features. RFE starts with all of the features and then eliminates features one by one (based on their importance). It stops when the desired number of features are left, or when the elimination of features no longer helps the model.

In [ ]:
# Replace N/A with the most frequent value in that feature

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_corw)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_corw), columns = X_train_corw.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_corw), columns = X_valid_corw.columns)

In [ ]:
# Standardize each feature to have mean 0 and standard deviation 1, this will help the model fitting

scaler = StandardScaler()
scaler.fit(X_imputed_train)

X_imputed_train_std = pd.DataFrame(scaler.transform(X_imputed_train), columns=X_imputed_train.columns)
X_imputed_valid_std = pd.DataFrame(scaler.transform(X_imputed_valid), columns=X_imputed_valid.columns)

del X_imputed_train
del X_imputed_valid

In [ ]:
f1_scorer = make_scorer(
    f1_score, 
    greater_is_better=True, 
    )


rfecv = RFECV(
    estimator=LogisticRegression(random_state = 175, 
                                 class_weight='balanced', 
                                 C=200, 
                                 dual=False, 
                                 solver='liblinear'
                                ),
    cv=StratifiedKFold(2),
    scoring =  f1_scorer,
    step = 1,
    n_features_to_select = None,
    min_features_to_select = 1,
    n_jobs = -1,
)

rfecv.fit(X_imputed_train_std, y_train.values.ravel())
rfecv.show() 

In [ ]:
mask = rfecv.get_support()
columns = X_imputed_train_std.columns
selected_cols = columns[mask]
n_features5 = len(selected_cols)
X_train_rfe = pd.DataFrame(rfecv.transform(X_imputed_train_std.values), columns = selected_cols)
X_valid_rfe = pd.DataFrame(rfecv.transform(X_imputed_valid_std.values), columns = selected_cols)

print(f'From {len(X_imputed_train_std.columns)} features, we are left with {n_features5} features')
n_features.append(n_features5)

#### Evaluate the effect of the recursive feature elimination

In [ ]:
time, f1score, roc_aucscore = evaluate(train_df = X_train_rfe, 
                                       test_df = X_valid_rfe, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )


print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')

f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
del time
del f1score
del roc_aucscore

### I.6. Select from model
We can also select features based on the feature importance ranking reported by a classifier. 

In [ ]:
from sklearn.feature_selection import SelectFromModel

clf = RandomForestClassifier(
    n_estimators = 100, 
    criterion = 'entropy',
    max_depth = 5,
    min_samples_split = 5,
    random_state = 180,
    n_jobs = -1,
)

clf = clf.fit(X_train_rfe, y_train.values.ravel())

model = SelectFromModel(clf, prefit=True)
feature_idx = model.get_support()
feature_name = X_train_rfe.columns[feature_idx]
n_features6 = len(feature_name)

X_train_RF = pd.DataFrame(model.transform(X_train_rfe.values), columns=feature_name)
X_valid_RF = pd.DataFrame(model.transform(X_valid_rfe.values), columns=feature_name)

print(f'From {len(X_imputed_train_std.columns)} features, we are left with {n_features6} features')
n_features.append(n_features6)

#### Evaluate the effect of the Select from Model elimination

To keep the comparison similar, we train a logistic regression model on the features selected by the RF model, rather than using the score of the RF model itself. 

In [ ]:
time, f1score, roc_aucscore = evaluate(train_df = X_train_RF, 
                                       test_df = X_valid_RF, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )

print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
del time
del f1score
del roc_aucscore

### Results of the feature selection

We went down from 589 features to just 12 features. The training time clearly goes down drastically. The information loss is also considerable, especially from step 4 onwards, as seen through the f1 score. That step obviously removes too many features, but it was good enough for the purpose of this demo. 
>For your own curiosity, modify the parameter we used in step 4 for the correlation between the features and the target, from 0,05 to 0,03 or smaller. What do you expect to see in this step and in the next?

## Feature extraction

Whereas in feature selection the focus was on deciding which features of the original data to retain, in feature extraction (or feature engineering), the focus is on identifying a smaller number of (possibly new) features that retain as much as possibly the "quality" of the dataset. These new features depend on the original features, but they can be very different from them. 

We start the demo of the feature extraction from the 'X_train_nosparse' version of the data, i.e., that where we just removed the columns missing more than 50% of the data. 

We imputate the missing values, standardize the data, and proceed with the demo. 

In [ ]:
# Replace N/A with the most frequent value in that feature

imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train_nosparse)

X_imputed_train = pd.DataFrame(imputer.transform(X_train_nosparse), columns = X_train_nosparse.columns)
X_imputed_valid = pd.DataFrame(imputer.transform(X_valid_nosparse), columns = X_valid_nosparse.columns)

# Standardize each feature to have mean 0 and standard deviation 1, this will help the model fitting
scaler = StandardScaler()
scaler.fit(X_imputed_train)

X_train = pd.DataFrame(scaler.transform(X_imputed_train), columns=X_imputed_train.columns)
X_valid = pd.DataFrame(scaler.transform(X_imputed_valid), columns=X_imputed_valid.columns)

### II.1 PCA

In [ ]:
# Apply PCA

pca = PCA().fit(X_train.values)
pca_var_df = pd.DataFrame({'Information': pca.explained_variance_ratio_,
                           'PCs': np.arange(pca.n_components_)
                          })
plt.figure(figsize = (25,8))
sns.barplot(x = 'PCs',y = 'Information',data = pca_var_df)

In [ ]:
# Find the number of principal components required for an explained variance of at least 80%

var = 0
for i in range(pca.n_components_): 
    var += pca.explained_variance_ratio_[i]
    if var > 0.8:
        break
        
n_comp_80var = i+1
print('For an 80% variance the number of PCs we need is', n_comp_80var)

pca_var_df = pd.DataFrame({'Information': pca.explained_variance_ratio_[0:n_comp_80var],
                           'PCs': np.arange(n_comp_80var)
                          })
plt.figure(figsize = (25,8))
sns.barplot(x = 'PCs',y = 'Information',data = pca_var_df)

#### Use PCA for visualization

In [ ]:
# Extract the first 2 principal components
X_train_2comps = PCA(n_components = 2).fit_transform(X_train)

fig = plt.figure(figsize = (10,7))
plt.plot(X_train_2comps[:,0], X_train_2comps[:,1], marker = 'o', color = 'teal', alpha = .5, linewidth = 0)
plt.xlabel('Principal Component 1', fontsize = 14)
plt.ylabel('Principal Component 2', fontsize = 14)
plt.grid(linestyle = '--')

In [ ]:
# Colour code the 2 Principal Component plot depending on whether target is 1 or 0

# Store binary y and X in a dataframe 
plot_2comps = pd.DataFrame(X_train_2comps, columns = ['PC1', 'PC2'])

plot_2comps['Pass'] = y_train.to_numpy()

X_2comps_0 = plot_2comps[plot_2comps['Pass'] == 0][['PC1', 'PC2']].copy()
X_2comps_1 = plot_2comps[plot_2comps['Pass'] == 1][['PC1', 'PC2']].copy()

fig = plt.figure(figsize = (10,7))
plt.plot(X_2comps_0['PC1'], X_2comps_0['PC2'], marker = 'o', color = 'red', alpha = .5,
         linewidth = 0, label = 'Pass = 0')
plt.plot(X_2comps_1['PC1'], X_2comps_1['PC2'], marker = 'o', color = 'blue', alpha = .5,
         linewidth = 0, label = 'Pass = 1')
plt.xlabel('Principal Component 1', fontsize = 14)
plt.ylabel('Principal Component 2', fontsize = 14)
plt.legend()
#plt.grid(linestyle = '--')

NOTE: the data is not well separated by the 2 PCs. This is likely because they retain too little of the variance of the data (recall we needed all the 81 first PCs to retain 80% of the variance). 

Run the cell about with only one of the classes displayed so you can realise where they are placed in this 2D plot. 

#### Use PCA for dimensionality reduction for other models to train on

In [ ]:
# Project the data onto the PCs that offer 80% variance

pca80 = PCA(n_components=n_comp_80var).fit(X_train.values)
pca80.fit(X_train.values)

X_train_pca80 = pd.DataFrame(pca80.transform(X_train.values))
X_valid_pca80 = pd.DataFrame(pca80.transform(X_valid.values))

time, f1score, roc_aucscore = evaluate(train_df = X_train_pca80, 
                                       test_df = X_valid_pca80, 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
n_features.append(pca80.n_components_)
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
del time
del f1score
del roc_aucscore

### II.2 LDA

#### Use LDA for visualization

In [ ]:
lda = LDA()
lda.fit(X_train, y_train.values.ravel())

print(lda.explained_variance_ratio_)

X_train_lda = lda.transform(X_train)
X_valid_lda = lda.transform(X_valid)

plot_lda = pd.DataFrame(X_train_lda, columns = ['LDA'])
plot_lda['Pass'] = y_train.to_numpy()

plot_lda_0 = plot_lda[plot_lda['Pass'] == 0][['LDA']].copy()
plot_lda_1 = plot_lda[plot_lda['Pass'] == 1][['LDA']].copy()


plt.figure(figsize=(10,2))
plt.scatter( plot_lda_0, [0] * plot_lda_0.shape[0], marker = 'o', color = 'red', alpha = .5,
         linewidth = 1, label = 'Pass = 0')
plt.scatter( plot_lda_1, [0] * plot_lda_1.shape[0], marker = 'o', color = 'blue', alpha = .5,
         linewidth = 1, label = 'Pass = 1')
plt.legend()
plt.show()

#### Use LDA for dimensionality reduction for other models to train on

In [ ]:
time, f1score, roc_aucscore = evaluate(train_df = pd.DataFrame(X_train_lda), 
                                       test_df = pd.DataFrame(X_valid_lda), 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
n_features.append(lda.explained_variance_ratio_.shape[0])
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
del time
del f1score
del roc_aucscore

### II.3 t-SNE
t-Distributed Stochastic Neighbor Embedding (t-SNE) is a technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. Points that are close to the original feature space are also expressed in a two-dimensional plane after compression. Since the nonlinear relationship can be identified, the model performance can be improved by adding the compression results expressed by these t-SNEs to the original features. However, since the computation cost is high, it is not suitable for compression exceeding two or three dimensions.

>Link to the author Laurens van der Maaten's page (it includes links to his original paper, to lectures and implementations): https://lvdmaaten.github.io/tsne/

>Link to a simplified presentation of the main ideas in t-SNE: https://youtu.be/NEaUSP4YerM?si=239qIfc12m7SCFS4

In [ ]:
tsne = TSNE(
    n_components = 2,
    perplexity = 5,
)

X_train_tsne = tsne.fit_transform(X_train)
X_valid_tsne = tsne.fit_transform(X_valid)

# Colour code the t-SNE plot depending on whether target is 1 or 0

# Store binary y and X in a dataframe 
plot_tsne = pd.DataFrame(X_train_tsne, columns = ['tSNE1', 'tSNE2'])

plot_tsne['Pass'] = y_train.to_numpy()

X_train_tsne_0 = plot_tsne[plot_tsne['Pass'] == 0][['tSNE1', 'tSNE2']].copy()
X_train_tsne_1 = plot_tsne[plot_tsne['Pass'] == 1][['tSNE1', 'tSNE2']].copy()

fig = plt.figure(figsize = (10,7))
plt.plot(X_train_tsne_0['tSNE1'], X_train_tsne_0['tSNE2'], marker = 'o', color = 'red', alpha = .5,
         linewidth = 0, label = 'Pass = 0')
plt.plot(X_train_tsne_1['tSNE1'], X_train_tsne_1['tSNE1'], marker = 'o', color = 'blue', alpha = .5,
         linewidth = 0, label = 'Pass = 1')
plt.xlabel('t-SNE Component 1', fontsize = 14)
plt.ylabel('t-SNE Component 2', fontsize = 14)
plt.legend()
#plt.grid(linestyle = '--')

#### Use t-SNE for dimensionality reduction for other models to train on

In [ ]:
time, f1score, roc_aucscore = evaluate(train_df = pd.DataFrame(X_train_tsne), 
                                       test_df = pd.DataFrame(X_valid_tsne), 
                                       train_target=y_train, 
                                       test_target=y_valid
                                      )
print(f' Training time: {time}ms\n F1 Score: {f1score}\n ROC-AUC Score: {roc_aucscore}')
n_features.append(tsne.embedding_.shape[1])
f1scores.append(f1score)
roc_aucscores.append(roc_aucscore)
times.append(time)
del time
del f1score
del roc_aucscore

### Summary of feature selection and feature engineering: how did we do?

In [ ]:
xlabels = ['Full', 'NoSparse', 'NoVar', 'NoPairCorr', 'NoTarCorr', 'RFE', 'Model', 'PCA', 'LDA', 'tSNE']

fig, (ax2, ax0, ax1, ax4) = plt.subplots(4, 1, figsize=(8, 8))
fig.tight_layout()

ax2.plot(n_features, label='# of features', c='g')
ax2.set(ylabel='# of features')
#ax2.set(xlabel='Feature selection step')
ax2.set_xticks(np.arange(len(n_features)), labels=xlabels)
ax2.legend()

ax0.plot(times, label='Training time', c='b')
ax0.set(ylabel='Training Time (ms)')
ax0.set_xticks(np.arange(len(times)), labels=xlabels)
ax0.legend()

ax1.plot(f1scores, label='F1 Score', c='r')
ax1.set(ylabel='F1 score')
#ax1.set(xlabel='Feature selection step')
ax1.set_xticks(np.arange(len(f1scores)), labels=xlabels)
ax1.set_ylim([0.4, 1])
ax1.legend()

ax4.plot(roc_aucscores, label='ROC-AUC Score', c='m')
ax4.set(ylabel='ROC-AUC score')
#ax4.set(xlabel='Feature selection step')
ax4.set_xticks(np.arange(len(roc_aucscores)), labels=xlabels)
ax4.set_ylim([0.4, 1])
ax4.legend()

#fig.show()

**Conclusion.** We have a clear winner on this dataset: LDA produced a single engineered feature that offers a model of the same quality as the full set of almost 600 features! Note though that the performance of the model on the minority class remained limited throughout our attempts. 

**Bonus challenge.** Send me any model (using any architecture and any data processing) for the UCI SECOM dataset with an ROC_AUC score over 0.7 on the test dataset, for an extra 10 points under "Assignments". 

# Challenge: Apply feature selection and engineering to the Penguin dataset
The dataset only has 7 features and it can of course be used as is for machine learning. It is still worth it to do feature selection/engineering for visualization purposes: see how the data is distributed in 2 well-chosen dimensions. 

## Meet the penguins (Artwork by @allison_horst https://github.com/allisonhorst/penguins)

![](https://imgur.com/orZWHly.png)

The Palmer Archipelago (Antarctica) penguin dataset is great for data exploration & visualization, as an alternative to iris. Data were collected and made available by Dr. Kristen Gorman and the Palmer Station, Antarctica LTER, a member of the Long Term Ecological Research Network. Please see https://github.com/allisonhorst/palmerpenguins for more information.

The dataset consists of 7 columns. 

* **species**: penguin species (Chinstrap, Adélie, or Gentoo)
* **culmen_length_mm**: culmen length (mm)
* **culmen_depth_mm**: culmen depth (mm)
* **flipper_length_mm**: flipper length (mm)
* **body_mass_g**: body mass (g)
* **island**: island name (Dream, Torgersen, or Biscoe) in the Palmer Archipelago (Antarctica)
* **sex**: penguin sex

<img src="https://i.imgur.com/trfGnPa.png" width="800px">

## Penguin body parts: flippers, culmen (Artwork by @allison_horst https://github.com/allisonhorst/penguins)

<img src="https://i.imgur.com/ie2WP7w.jpg" width="800px">

### Part 1: data analytics
- Load the dataset from OpenML (ID 42585). Check the X and the y parts returned by the function call. In this notebook we will consider this as a classifier with the target being the species name. We will do both visualization, as well as training classifiers. 
- Q1: Do you consider this data to be imbalanced? 
- Q2: How many features do you have (consider the target as a separate column, not part of the features)? 
- Q3: How many features contain missing values? 
- Check the 'sex' feature with penguin_X['sex'].value_counts(). It contains a datapoint with 'sex' set to a value that does not make sense. Remove that value (change it to appear as a missing value) with the call penguin_X['sex'] = penguin_X['sex'].cat.remove_categories(['WRITE_WEIRD_VALUE_HERE']) 

In [ ]:
# Your code here


In [ ]:
# Encode the categorical features 'island' and 'sex' with numerical encodings.
# 'sex' has N/A values, and they should remain N/A also after the change in the encoding.
# One solution is to save the rows with N/A, transform the others, then put back the rows with N/A.

from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

penguin_X['island'] = lb.fit_transform(penguin_X['island'])

peng_na = penguin_X[penguin_X['sex'].isna()].copy()
penguin_X.dropna(subset='sex', axis=0, inplace=True)
penguin_X['sex'] = lb.fit_transform(penguin_X['sex'])

penguin_X = pd.concat([penguin_X, peng_na])
del peng_na

penguin_y = pd.DataFrame(lb.fit_transform(penguin_y))

penguin_X.info()

### Part 1: feature selection/engineering

- Split your dataset into train/validation/test using the same percentages as in the SECOM dataset, using a stratified approach or not, depending on your answer to Q1. 
- Q4: Did you use a stratified split? 
- Imputate the missing values in your train/valid datasets using the most frequent-vased strategy. 
- Standardize the train/valid datasets using a Standard Scaler. 

In [ ]:
# Your code here


Use the code below to evaluate the effects of the feature selection/engineering methods. This is slightly different than the SECOM dataset, because we have a 3-class classification dataset. 

In [ ]:
def evaluate2(train_df, test_df, train_target, test_target):
    
    # training the model
    logreg = LogisticRegression(
        random_state = 175, 
        class_weight='balanced', 
        C=200, 
        dual=False, 
        multi_class='multinomial'
    )
    start_time = datetime.datetime.now()
    logreg.fit(train_df, train_target.values.ravel())
    elapsed = datetime.datetime.now() - start_time
    time = int(elapsed.total_seconds()*1000)
    
    logreg.class_counts_ = train_target.nunique()
    
    # evaluation and scoring
    y_pred_prob = logreg.predict_proba(test_df)
    y_pred = logreg.predict(test_df)
    y_true = test_target.values.ravel()
    f1score = f1_score(y_true, y_pred, average='micro')
    roc_aucscore = roc_auc_score(y_true, y_pred_prob, average='micro', multi_class='ovr')
    
    # visualizations
    cre = ClassPredictionError(
        logreg, 
        isfitted=True,
        classes=['Adelie', 'Chinstrap', 'Gentoo'],
        label_encoder={0: 'Adelie', 1: 'Chinstrap', 2:'Gentoo'},
        size=(400, 400)
    )
    cre.score(test_df, y_true)
    cre.show()
    cm = ConfusionMatrix(
        logreg, 
        isfitted=True,
        percent= True,
        size=(300, 300)
    )
    
    cm.score(test_df, y_true)
    cm.show()
    
    return time, f1score, roc_aucscore


Use the standardized datasets in each of the evaluation to be done. 
- Evaluate the learnability of the dataset with all its features in.
- Do feature selection based on a random forest model (code below). 
- Q4: Which features did the random forest select? 
- Evaluate the learnability of the dataset projected on the features selected by the RF model. 
- Apply PCA with as few components you need in order to preserve at least 80% of the data variance
- Q5: How many PCs did you need? 
- Evaluate the learnability of the dataset transformed through the PCA you just trained. 
- Visualize the dataset along the first two PCs.
- Apply LDA and visualize the data along the 2 components you get. 
- Evaluate the learnability of the dataset transformed through the LDA you just trained. 
- Apply t-SNE with 2 components and visualize the data along the 2 components you get. 
- Evaluate the learnability of the dataset transformed through the t-SNE you just trained. 
- Plot the results of the 5 evaluations you did. 

Use the code below for the model-based feature selection with random forests. 

In [ ]:
from sklearn.feature_selection import SelectFromModel

clf = RandomForestClassifier(
    n_estimators = 100, 
    criterion = 'entropy',
    max_depth = 5,
    min_samples_split = 5,
    random_state = 180,
    n_jobs = -1,
)

clf = clf.fit(X_train_std, y_train.values.ravel())

model = SelectFromModel(clf, prefit=True)
feature_idx = model.get_support()
feature_name = X_train_std.columns[feature_idx]
n_features6 = len(feature_name)

Visualization code for PCA.

In [ ]:
# Extract the first 2 principal components
X_train_2comps = PCA(n_components = 2).fit_transform(X_train_std)

# Colour code the 2 Principal Component plot

# Store binary y and X in a dataframe 
plot_2comps = pd.DataFrame(X_train_2comps, columns = ['PC1', 'PC2'])

plot_2comps['species'] = y_train.to_numpy()

X_2comps_0 = plot_2comps[plot_2comps['species'] == 0][['PC1', 'PC2']].copy()
X_2comps_1 = plot_2comps[plot_2comps['species'] == 1][['PC1', 'PC2']].copy()
X_2comps_2 = plot_2comps[plot_2comps['species'] == 2][['PC1', 'PC2']].copy()

fig = plt.figure(figsize = (10,10))
plt.plot(X_2comps_0['PC1'], X_2comps_0['PC2'], marker = 'o', color = 'red', alpha = .5,
         linewidth = 0, label = 'Adelie')
plt.plot(X_2comps_1['PC1'], X_2comps_1['PC2'], marker = 'o', color = 'blue', alpha = .5,
         linewidth = 0, label = 'Chinstrap')
plt.plot(X_2comps_2['PC1'], X_2comps_2['PC2'], marker = 'o', color = 'green', alpha = .5,
         linewidth = 0, label = 'Gentoo')
plt.xlabel('Principal Component 1', fontsize = 14)
plt.ylabel('Principal Component 2', fontsize = 14)
plt.legend()
#plt.grid(linestyle = '--')

Visualization code for LDA.

In [ ]:
lda = LDA()
lda.fit(X_train_std, y_train.values.ravel())

print(lda.explained_variance_ratio_)

X_train_lda = lda.transform(X_train_std)
X_valid_lda = lda.transform(X_valid_std)

plot_lda = pd.DataFrame(X_train_lda, columns = ['LDA1',  'LDA2'])
plot_lda['species'] = y_train.to_numpy()

plot_lda_0 = plot_lda[plot_lda['species'] == 0][['LDA1', 'LDA2']].copy()
plot_lda_1 = plot_lda[plot_lda['species'] == 1][['LDA1', 'LDA2']].copy()
plot_lda_2 = plot_lda[plot_lda['species'] == 2][['LDA1', 'LDA2']].copy()


plt.figure(figsize=(10,10))
plt.scatter( plot_lda_0['LDA1'], plot_lda_0['LDA2'], marker = 'o', color = 'red', alpha = .5,
         linewidth = 1, label = 'Adelie')
plt.scatter( plot_lda_1['LDA1'], plot_lda_1['LDA2'], marker = 'o', color = 'blue', alpha = .5,
         linewidth = 1, label = 'Chinstrap')
plt.scatter( plot_lda_2['LDA1'], plot_lda_2['LDA2'], marker = 'o', color = 'green', alpha = .5,
         linewidth = 1, label = 'Gentoo')
plt.legend()
plt.show()

Visualization code for t-SNE.

In [ ]:
tsne = TSNE(
    n_components = 2,
    perplexity = 5,
)

X_train_tsne = tsne.fit_transform(X_train_std)
X_valid_tsne = tsne.fit_transform(X_valid_std)

# Colour code the t-SNE plot depending on the target

# Store binary y and X in a dataframe 
plot_tsne = pd.DataFrame(X_train_tsne, columns = ['tSNE1', 'tSNE2'])

plot_tsne['species'] = y_train.to_numpy()

X_train_tsne_0 = plot_tsne[plot_tsne['species'] == 0][['tSNE1', 'tSNE2']].copy()
X_train_tsne_1 = plot_tsne[plot_tsne['species'] == 1][['tSNE1', 'tSNE2']].copy()
X_train_tsne_2 = plot_tsne[plot_tsne['species'] == 2][['tSNE1', 'tSNE2']].copy()

fig = plt.figure(figsize = (10,10))
plt.plot(X_train_tsne_0['tSNE1'], 
         X_train_tsne_0['tSNE2'], 
         marker = 'o', 
         color = 'red', 
         alpha = .5,
         linewidth = 0, 
         label = 'Adelie'
        )
plt.plot(X_train_tsne_1['tSNE1'], 
         X_train_tsne_1['tSNE1'], 
         marker = 'o', 
         color = 'blue', 
         alpha = .5,
         linewidth = 0, 
         label = 'Chinstrap'
        )
plt.plot(X_train_tsne_2['tSNE1'], 
         X_train_tsne_2['tSNE1'], 
         marker = 'o', 
         color = 'green', 
         alpha = .5,
         linewidth = 0, 
         label = 'Gentoo'
        )
plt.xlabel('t-SNE Component 1', fontsize = 14)
plt.ylabel('t-SNE Component 2', fontsize = 14)
plt.legend()
#plt.grid(linestyle = '--')

Visualization code for the overall evaluation results. 

In [ ]:
xlabels = ['Full', 'RF', 'PCA', 'LDA', 'tSNE']

fig, (ax1, ax4) = plt.subplots(2, 1, figsize=(8, 4))
fig.tight_layout()

ax1.plot(f1scores, label='F1 Score', c='r')
ax1.set(ylabel='F1 score')
#ax1.set(xlabel='Feature selection step')
ax1.set_xticks(np.arange(len(f1scores)), labels=xlabels)
ax1.set_ylim([0.4, 1])
ax1.legend()

ax4.plot(roc_aucscores, label='ROC-AUC Score', c='m')
ax4.set(ylabel='ROC-AUC score')
#ax4.set(xlabel='Feature selection step')
ax4.set_xticks(np.arange(len(roc_aucscores)), labels=xlabels)
ax4.set_ylim([0.4, 1])
ax4.legend()

#fig.show()

- Q6: Did feature selection/engineering help produce a dataset that is drastically easier to learn? 
- Q7: Which of PCA, LDA, t-SNE gave the weakest visual separation between the 3 classes? 
- Q8: Which of random forests, PCA, LDA, t-SNE gave the weakest dataset for classification purposes? 